# PDF yüklemesinin gerçekleştirilmesi

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("basketbol_kural.pdf")
data = loader.load()

In [2]:
len(data)

97

# Veriyi parçalara ayırma (Chunking İşlemi)

Tek bir büyük metin parçası yerine, dokümantasyonu daha küçük parçalara böldük? Doğru bilgiyi daha kolay bulmasını sağlayabilmek için bu işlemi gerçekleştirdik.
Metni 1000 karakterlik parçalar halinde bölerek işlemeye hazır hale getiriyoruz.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total", len(docs))

Total 273


In [4]:
docs[7]

Document(metadata={'producer': 'Adobe PDF Library 22.1.174', 'creator': 'Word için Acrobat PDFMaker 22', 'creationdate': '2022-10-10T16:05:00+03:00', 'author': 'Murat Biricik', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2022-10-10T16:05:36+03:00', 'sourcemodified': 'D:20221010130433', 'subject': '', 'title': '', 'source': 'basketbol_kural.pdf', 'total_pages': 97, 'page': 4, 'page_label': '5'}, page_content='BASKETBOL OYUN KURALLARI \n \n \nEkim 2022 v.01 TÜRKİYE BASKETBOL FEDERASYONU 3 \n \n MADDE 35 ÇİFT FAUL …………………………………………………………………………………………………………………..42  \n MADDE 36 TEKNİK FAUL ………………………………………………………………………………………………………………43  \n MADDE 37 SPORTMENLİK DIŞI FAUL ......……………………………………………………………………………………….44 \n MADDE 38 DİSKALİFİYE EDİCİ FAUL ...…...…...…………………………………………………………………………………45  \n MADDE 39 KAVGA ..……......………………………………………………………………………………………………………….47  \nKURAL YEDİ – GENEL KOŞULLAR ......…………………………………………………………………………………………………………..48  \n MADDE 40 BİR OYUNCUNUN 5 FAULÜ ……

# Google Generative AI Embeddings'i kullanarak Embedding oluşturma kısmı

Text veriyi sayısal bir veriye çevirmeye çalıştık.

In [5]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello")
vector[:5]

[0.017157701775431633,
 -0.009304771199822426,
 -0.011424741707742214,
 -0.045027956366539,
 0.029878143221139908]

# ChromaDB üzerine kayıtı işlemi

In [6]:
vectorstore = Chroma.from_documents(documents=docs,embedding=embeddings)

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs= {"k" : 10})  

In [9]:
retrieved_docs = retriever.invoke("Arkadan yapılan kural dışı savunma nedir?")

In [10]:
len(retrieved_docs)

10

In [11]:
#print(retrieved_docs[5].page_content)

kabul edilir. 
16.2 Kural 
16.2.1 Top, hücum edilen rakibin sepetine girdiğinde aşağıdaki gibi sayı kaydedilir: 
• Serbest atıştan yapıldığında 1 sayı 
• 2 sayılık bölgeden yapıldığında 2 sayı 
• 3 sayılık bölgeden yapıldığında 3 sayı 
• Son serbest atışta top çembere temas ettikten sonra ve sepete girmeden önce, bir hücum ya 
da savunma oyuncusu tarafından kurallara uygun olarak topa temas edildiğinde 2 sayı sayılır. 
16.2.2 Bir oyuncu kazara/istem dışı olarak kendi sepetine sayı yaparsa 2 sayı verilir ve rakip takımın 
sahadaki kaptanı yapmış gibi kaydedilir. 
16.2.3 Bir oyuncu bilerek kendi sepetine sayı yaparsa bu bir ihlaldir ve sayı geçerli sayılmaz. 
16.2.4 Bir oyuncu topun tamamının sepete alttan girmesine neden olursa, bu bir ihlaldir.  
16.2.5 Bir oyuncunun topun oyuna sokulmasında ya da son serbest atış sonrasındaki ribauntta topun 
kontrolünü sağlaması ve bir atış girişiminde bulunabilmesi için oyun saati 0.3 (saniyenin onda


# Google Gemini API yapısını kullanarak LLM Invoke işlemleri

In [12]:
#Temperature Değer Aralığı

#Düşük Değerler(0.1-0.4):Daha kesin ve daha tutarlı cevaplar vermemizi sağlayabilir. Model daha tahmin edilebilir hale gelir.
#Orta Değerler(0.4-0.7): Hem mantıklı hem de yaratıcı cevaplar verebiliyor.\n
#Yüksek Değerler(0.8-1.0) : Daha rastgele ve yaratıcı cevaplar ve bazen de belki tutarsız cevaplar verebilir.

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=500
)

In [14]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [15]:
system_prompt = (
    "Sen bir yardımcı asistansın ve yalnızca basketbol kuralları hakkında sorulara cevap veriyorsun. "
    "Yanıtlarını yalnızca verilen bağlam içeriğinden oluştur. "
    "Eğer sorunun cevabını bilmiyorsan, 'Bu konuda yardımcı olamıyorum.' de. "
    "Cevaplarını en fazla üç cümle ile ver ve doğru bilgi içerdiğinden emin ol.\n\n"
    "{context}"
)

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

# Soru-Cevap Üretme Zinciri Oluşturu (LLM+PROMPT)

In [17]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

# RAG Zinciri Oluşturma ( RAG + LLM Entegrasyonun Gerçekleşmesi )

In [18]:
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [21]:
# Kullanıcı Sorgusunu Çalıştırarak Cevap Üretme İşlemi

In [19]:
response = rag_chain.invoke({"input":"Arkadan yapılan kural dışı savunma nedir?"})

In [20]:
print(response)

{'input': 'Arkadan yapılan kural dışı savunma nedir?', 'context': [Document(id='e02041c6-5fd3-471c-9aac-46ba1f0b6392', metadata={'author': 'Murat Biricik', 'comments': '', 'company': '', 'creationdate': '2022-10-10T16:05:00+03:00', 'creator': 'Word için Acrobat PDFMaker 22', 'keywords': '', 'moddate': '2022-10-10T16:05:36+03:00', 'page': 57, 'page_label': '58', 'producer': 'Adobe PDF Library 22.1.174', 'source': 'basketbol_kural.pdf', 'sourcemodified': 'D:20221010130433', 'subject': '', 'title': '', 'total_pages': 97}, page_content='Madde 47  Hakemler: Görevleri ve yetkileri \n47.1 Hakemler, hakem masası, takım sıraları ve çizgilerin hemen gerisindeki zemin alanları da dahil \nolmak üzere, sınır çizgileri içinde ya da dışında, kurallara aykırı olan konularda karar verme \nyetkilerine sahip olacaktır. \n47.2 Hakemler kurallara aykırı bir durum olduğunda, bir çeyrek ya da uzatma sona erdiğinde ya da \noyunu durdurmaya gerekli gördüklerinde düdüklerini çalacaklardır. Hakemler, sahadan yap

In [21]:
print(response["answer"]);

Arkadan yapılan kural dışı savunma, bir savunma oyuncusu tarafından bir rakibe arkadan yapılan bir kişisel temastır. Savunma oyuncusunun topla oynama girişimi, rakibine arkadan yaptığı temasın haklılığını göstermez.
